In [ ]:
# ALL THE PACKAGES NEEDED TO BE INSTALLED ARE IN THE LINE OF CODE BELOW
!pip install tenseal torch torchvision pillow numpy pandas scikit-learn
!pip list

In [ ]:
# THIS IS WHERE THE IMPORTATION OF THE NECESSARY PCKAGES COMES TO PLAY

import os
import numpy as np
import pandas as pd
import tenseal as ts
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
# SETTING THE VARIABLES FOR THE TRAIN AND TEST DATA RESPECTIVELY

torch.manual_seed(42)
np.random.seed(42)

DATA_DIR = "./chest_xray"  # THIS DATASET WAS DOWNLOADED TO LOCAL DRIVE FOR FASTER EXECUTION(Directly using the LOCAL Hardware), AS COMPARED TO GOOGLE DRIVE
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [ ]:
# SINCE WE ARE DEALING WITH ENCRYPTION, THERE IS A NEED FOR US TO GET SOME IMPORTANT KEYS
# INCLUDING THE SECRET KEYS AND WE WILL ALSO USE IT FOR THE ENCYPTION OF THE IMAGES VIA THE TENSEAL PACKAGE BY  CREATING THIS FUNCTION BELOW
def create_stable_context():
    """Context configuration that guarantees stability"""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=16384,  
        coeff_mod_bit_sizes= [60, 40, 40, 40, 60]
        # coeff_mod_bit_sizes= [60, 40, 40, 60]
        # coeff_mod_bit_sizes=[40, 21, 21, 40]  
    )
    context.generate_galois_keys()
    context.global_scale = 2**21  
    return context



# THIS IS THE SECTION WHERE WE WORK ON THE DATASET(DATA PREPARATION AND VALIDATION) AND PREPARE IT FOR THE NEXT TRAINNG OPERATION 
class SafePneumoniaDataset:
    def __init__(self, data_dir, context, img_size=64):
        self.context = context
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.0], std=[4.0]) 
        ])
        self.image_paths = []
        self.labels = []
        
        for class_dir in ["NORMAL", "PNEUMONIA"]:
            dir_path = os.path.join(data_dir, class_dir)
            for img_name in os.listdir(dir_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(dir_path, img_name))
                    self.labels.append(0 if class_dir == "NORMAL" else 1)
    
    def get_safe_batches(self, batch_size=1):  
        for i in range(0, len(self.image_paths), batch_size):
            img_path = self.image_paths[i]
            img = Image.open(img_path).convert('L')
            img = self.transform(img)
            # Double safety scaling
            encrypted = ts.ckks_vector(self.context, img.numpy().flatten() * 0.125)
            yield encrypted, self.labels[i]


# creating the Polynomial Activation Function
class PolynomialActivation:
    def __init__(self, a=0.25, b=0.5, c=0.0):
        self.a = a
        self.b = b
        self.c = c

    def __call__(self, enc_vector):
        x = enc_vector
        x_sq = x * x
        return (x_sq * self.a) + (x * self.b) + self.c



# TO TRAIN A MODEL, WE NEED TO FIRST BUILD THE MODEL, WHICH IS WHAT WE HANDLED HERE.
class UltraSafeHEModel:
    def __init__(self, input_size, context):
        self.context = context
        self.weights = np.random.randn(input_size)
        self.bias = np.random.randn()
        self.activation = PolynomialActivation(a=0.125, b=0.5, c=0.25)  # Non-linear activation

    def ultra_safe_predict(self, encrypted_input):
        linear_result = encrypted_input.dot(self.weights) + self.bias
        return self.activation(linear_result)
    
    def ultra_safe_update(self, gradients):
        try:
            update = gradients * 0.00001  
            self.weights = self.weights - update
        except:
            print("Used nuclear-safe fallback update")
            pass  

    def predict_single(self, encrypted_img, threshold=0.5):
        try:
            output = self.ultra_safe_predict(encrypted_img)
            confidence = output.decrypt()[0]
            prediction = "PNEUMONIA" if confidence > threshold else "NORMAL"
            return prediction, float(confidence)
        except Exception as e:
            print(f"Prediction failed: {str(e)}")
            return "ERROR", 0.0
        


# THIS SECTION HANDLES THE DEFINITION OF THE FUNCTION RESPONSIBLE TO EXECUTE THE TRAINING OPERATION OF THE MODEL
def nuclear_train(model, dataset, epochs=10):
    print("I have started the training process")

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        processed = 0
        
        for img, label in dataset.get_safe_batches():
            output = model.ultra_safe_predict(img)
            
            try:
                error = (output * 0.5) - (float(label) * 0.5)
                loss = (error * 0.5) * (error * 0.5)  
                total_loss += loss.decrypt()[0]
                
                safe_grad = img * (error * 0.0625)  
                model.ultra_safe_update(safe_grad)
                
                pred = 1 if output.decrypt()[0] > 0.5 else 0
                correct += 1 if pred == label else 0
                processed += 1
            except:
                continue
        
        if processed > 0:
            avg_loss = total_loss / processed
            accuracy = 100 * correct / processed
            print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.8f} | Accuracy: {accuracy:.2f}%")


In [ ]:
# THIS SECTION HANDLES THE EXECUTION OF THE ABOVE DEFINED FUNCTIONS
CONTEXT = create_stable_context()

model = UltraSafeHEModel(64*64, CONTEXT)

train_data = SafePneumoniaDataset("./chest_xray/train", CONTEXT)

nuclear_train(model, train_data, epochs=10)


In [ ]:
# THIS IS WHERE THE EVALUATION OF THE MODEL COMES TO PLAY
def evaluate_model(model, test_dir, context, img_size=32):
    correct = 0
    true_pos = 0
    false_pos = 0
    false_neg = 0
    total = 0
    
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.0], std=[4.0])
    ])
    
    for class_dir in ["NORMAL", "PNEUMONIA"]:
        dir_path = os.path.join(test_dir, class_dir)
        for img_name in os.listdir(dir_path):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(dir_path, img_name)
                img = Image.open(img_path).convert('L')
                img = transform(img)
                encrypted_img = ts.ckks_vector(context, img.numpy().flatten() * 0.125)
                
                output = model.ultra_safe_predict(encrypted_img).decrypt()[0]
                pred = 1 if output > 0.5 else 0
                true_label = 0 if class_dir == "NORMAL" else 1
                
                total += 1
                correct += 1 if pred == true_label else 0
                
                if true_label == 1:
                    if pred == 1: true_pos += 1
                    else: false_neg += 1
                else:
                    if pred == 1: false_pos += 1
    
    accuracy = correct / total
    precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
    recall = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1

test_dir = "./chest_xray/test"
accuracy, precision, recall, f1 = evaluate_model(model, test_dir, CONTEXT)


In [ ]:
# ALL THE IMPORTANT KEYS NEED TO BE SAVED BECAUSE THEY ARE NEEDED TO MAKE THE ENCRYPTION AND PREDICTION
# THESE CODES BELOW IS TO ATTEND TO THE SAVING OF THE MODEL THAT HAVE BEEN TRAINED ABOVE
def save_he_models(model, context, save_dir="./pneumonia_he_model"):
    """
    Save all components needed to recreate the HE model
    Args:
        model: Your trained UltraSafeHEModel
        context: The TenSEAL context used for training
        save_dir: Directory to save components
    """
    import json
    import os
    
    os.makedirs(save_dir, exist_ok=True)
    
    # 1. Save the TenSEAL context (includes keys)
    with open(os.path.join(save_dir, "he_context.seal"), "wb") as f:
        f.write(context.serialize(save_secret_key=True))
    
    # 2. Save model weights (encrypted)
    with open(os.path.join(save_dir, "he_weights.seal"), "wb") as f:
        f.write(model.weights.serialize())
    
    # 3. Save model bias (encrypted)
    with open(os.path.join(save_dir, "he_bias.seal"), "wb") as f:
        f.write(model.bias.serialize())
    
    # 4. Save model metadata (JSON compatible)
    metadata = {
        "input_size": model.weights.size(),  # or model.weights.dimension(),  # Save the dimension only
        "model_type": "UltraSafeHEModel",
        "description": "Homomorphic Encrypted Pneumonia Classifier",
        "training_date": datetime.datetime.now().isoformat()
    }
    with open(os.path.join(save_dir, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent=4)
    
    print(f"Model successfully saved to {save_dir}")
    print(f"- Context: {save_dir}/he_context.seal")
    print(f"- Weights: {save_dir}/he_weights.seal")
    print(f"- Bias: {save_dir}/he_bias.seal")
    print(f"- Metadata: {save_dir}/metadata.json")


In [ ]:
save_he_models(model, CONTEXT)